In [1]:
! pip install keras-tuner

In [2]:
import tensorflow

In [3]:
from tensorflow import keras
import numpy as np

In [4]:
fashion_mnist = keras.datasets.fashion_mnist

In [5]:
(train_images,train_labels),(test_images,test_labels) = fashion_mnist.load_data()

In [6]:
##scaling the images between 0 to 1 for quick train our model
train_images = train_images/255.0
test_images = test_images/255.0

In [7]:
## reshaping our data
train_images = train_images.reshape(len(train_images),28,28,1)## 1 for grey scale channel
test_images = test_images.reshape(len(test_images),28,28,1)

In [12]:
## buildinh model layers
def model_building(hp):
    model = keras.Sequential([
        keras.layers.Conv2D(filters=hp.Int('conv_1_filter',min_value =64,
                                           max_value =128,step = 16),
                           kernel_size =hp.Choice('conv_1_kernel',values=[3,5]),
                            activation = 'relu',
                           input_shape = (28,28,1)),
        keras.layers.Conv2D(filters=hp.Int('conv_2_filter',min_value = 64,
                                          max_value=128,step=16),
                           kernel_size= hp.Choice('conv_2_kernel',values=[3,5]),
                           activation='relu'),
        keras.layers.Flatten(),
        keras.layers.Dense(units =hp.Int('dense_1_unit',min_value =32, max_value = 128,step=16),activation ='relu'),
        keras.layers.Dense(units =10,activation='softmax')
    ])
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',values =[10e-2,10e-3])),
                 loss='sparse_categorical_crossentropy',metrics = ['accuracy'])
    return model

In [13]:
from keras_tuner import RandomSearch
from keras_tuner.engine.hyperparameters import HyperParameters

In [14]:
tuner_search = RandomSearch(model_building,objective='val_accuracy',max_trials =2,
                           project_name = 'fashion_mnist_1')

In [15]:
tuner_search.search(train_images,train_labels,epochs=2,validation_split = 0.33)

Trial 2 Complete [00h 00m 18s]
val_accuracy: 0.09908590465784073

Best val_accuracy So Far: 0.8475329279899597
Total elapsed time: 00h 00m 39s
INFO:tensorflow:Oracle triggered exit


In [17]:
model = tuner_search.get_best_models(num_models=1)[0]

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 80)        2080      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 20, 20, 112)       224112    
_________________________________________________________________
flatten (Flatten)            (None, 44800)             0         
_________________________________________________________________
dense (Dense)                (None, 48)                2150448   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                490       
Total params: 2,377,130
Trainable params: 2,377,130
Non-trainable params: 0
_________________________________________________________________


In [19]:
model.fit(train_images,train_labels,epochs = 10,validation_split=0.1,initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.3822 - accuracy: 0.8589 - val_loss: 0.4168 - val_accuracy: 0.8487
Epoch 5/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.3625 - accuracy: 0.8665 - val_loss: 0.4159 - val_accuracy: 0.8542
Epoch 6/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.3475 - accuracy: 0.8732 - val_loss: 0.4291 - val_accuracy: 0.8528
Epoch 7/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.3260 - accuracy: 0.8789 - val_loss: 0.4406 - val_accuracy: 0.8597
Epoch 8/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.3248 - accuracy: 0.8803 - val_loss: 0.4234 - val_accuracy: 0.8483
Epoch 9/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.3010 - accuracy: 0.8884 - val_loss: 0.4282 - val_accuracy: 0.8602
Epoch 10/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.2936 - accuracy: 0.8928 - val_loss: 0.4989 - val_accuracy